In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os

import pandas as pd
import numpy as np
from string import Template
import pickle

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
def get_batting_data(years):
    
    '''
    Scrapes (using Selenium) batting performance data for individual players from batting statistics tables 
    from www.baseball-reference.com for input years.
    
    Import Requirements:
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    import time
    import os
    
    import pandas as pd
    import numpy as np
    from string import Template
    
    Input parameter:
    years: A list of integers, for example [2015, 2016 2017]
    
    Returns:
    List holding Pandas dataframes of batting performance data for each player, 
    one dataframe for each input year.
    '''
    
    chromedriver = "/Applications/chromedriver"
    os.environ["webdriver.chrome.driver"] = chromedriver
    driver = webdriver.Chrome(chromedriver)
    url_base = Template('https://www.baseball-reference.com/leagues/MLB/$year-standard-batting.shtml')
    df_list = []
    # Scrape batting data for each input year and append 
    for year in years:
        url = url_base.substitute(year=year)
        driver.get(url)
        # data tables are sourced to this page from other pages via javascript script
        tables = pd.read_html(driver.page_source)
        # individual player batting data is in the last (bottom) table on the page
        df = tables[len(tables) - 1]
        df_list.append(df)
        time.sleep(1);
    return df_list

def get_value_data():
    
    '''
    Scrapes (using Selenium) 2018 player salary data for individual players from www.baseball-reference.com.
    
    Import Requirements:
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    import time
    import os
    
    import pandas as pd
    import numpy as np
    
    Returns:
    Pandas dataframe including 2018 salaries for each individual player.
    '''
    
    chromedriver = "/Applications/chromedriver"
    os.environ["webdriver.chrome.driver"] = chromedriver
    driver = webdriver.Chrome(chromedriver)
    driver.get('https://www.baseball-reference.com/leagues/MLB/2018-value-batting.shtml')
    # data tables are sourced to this page from another page via javascript script
    tables = pd.read_html(driver.page_source)
    # individual salaries are in the last (bottom) table on the page
    df_value = tables[len(tables) - 1]
    return df_value

def pickle_3yr_data():
    
    '''
    Scrapes batting data for 2015, 2016 and 2017, and salary data for 2018, reads each into a Pandas
    dataframe and stores them in a pickle file.
    
    Import requirements:
    import pandas as pd
    import numpy as np
    import pickle
    
    Requires functions get_batting_data and get_value_data (from above) and the imports required for each
    '''
    
    # Get batting data for years 2015 - 2017 and read into list of dataframes, one for each year
    years = [2015, 2016, 2017]
    df_list = get_batting_data(years)
    
    # Get salary data for year 2018 and read into dataframe
    df_value = get_value_data()
    
    # Store list of batting dataframes and salary dataframe in pickle file
    pickle_object = (df_list, df_value)
    with open("batting_value_pickle.pkl", "wb") as f:
        pickle.dump(pickle_object, f)
    
def clean_salary_data():
    
    '''
    Cleans data in player salary dataframe preparing it for use as the target variable for a linear
    regression model.
    
    Import requirements:
    import pandas as pd
    import numpy as np
    import pickle
    
    Returns:
    Clean salary data.
    
    '''
    
    # Read source data from pickle file
    with open("batting_value_pickle.pkl", "rb") as f:
        pickle_object = pickle.load(f)
    df_value = pickle_object[1]
    
    # Remove white space from column names
    df_value.columns = [x.strip() for x in df_value.columns]
    
    # Drop columns except player name, player salary and player position and rename them
    drop_columns = ['Rk', 'Age', 'Tm', 'G', 'PA', 'Rbat', 'Rbaser', 'Rdp', 'Rfield', 'Rpos', 'RAA', 'WAA', 'Rrep',
                'RAR', 'WAR', 'waaWL%', '162WL%', 'oWAR','dWAR', 'oRAR','Acquired']
    df_salary = df_value.drop(drop_columns, axis=1)
    df_salary.columns = ['Name', 'salary', 'position']
    
    # Drop rows with any missing values
    df_salary.dropna(axis=0, how = 'any', inplace=True)
    
    # Remove rows containing column names (header rows from scraped data repeated to improve data view on web)
    df_salary = df_salary[df_salary['Name'] != 'Name']
    
    # Convert 'salary' string values to integer values
    df_salary['salary'] = df_salary['salary'].str.replace(',', '')
    df_salary['salary'] = df_salary['salary'].str.replace('$', '')
    df_salary['salary'] = df_salary['salary'].astype(int)
    
    # Remove players with pitcher position (defined as position '1' in data)
    df_salary = df_salary[~df_salary['position'].str.contains('1')]
    
    # Select players with salary greater than $1 million
    df_salary = df_salary[df_salary['salary'] > 1000000]
    
    # Drop 'position' column
    df_salary = df_salary.drop('position', axis=1)
    
    # Return clean dataframe
    return df_salary

def clean_batting_data(df):
    
    '''
    Cleans data in player batting dataframe preparing it for use as the feature data for a linear
    regression model.
    
    Import requirements:
    import pandas as pd
    import numpy as np

    Import parameter:
    A dataframe in the format delivered by function get_batting_data (from above)
    
    Returns:
    Clean batting data
    
    '''
    # Remove white space from column names
    df.columns = [x.strip() for x in df.columns]
    
    # Drop rows with any missing values
    df.dropna(axis=0, how = 'any', inplace=True)
    
    # Remove rows containing column names (header rows from scraped data repeated to improve data view on web)
    df = df[df['Name'] != 'Name']
    
    # Duplicate rows are included in the data table when a player played for more than one team during the year
    # The first row gives the total batting performance for the year for the player and following rows give
    # batting performance for each team.
    # Retain the first row with full year batting performance and drop other rows for the same player
    df = df.drop_duplicates(subset = 'Name', keep = 'first')
    
    # Select columns relevant to linear regression model analysis
    columns = ['Name', 'Age', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B',
       'HR', 'RBI', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'OPS', 'OPS+',
       'TB', 'GDP', 'HBP', 'SH', 'SF', 'IBB']
    df = df[columns]
    
    # Convert integer values to float values
    float_columns = ['Age', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B',
       'HR', 'RBI', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'OPS', 'OPS+',
       'TB', 'GDP', 'HBP', 'SH', 'SF', 'IBB']
    df[float_columns] = df[float_columns].astype('float')
    
    # Return clean dataframe
    return df

def combine_data():
    
    '''
    Merges individual batting performance dataframes for each of the three years included in this 
    analysis (2015, 2016, 2017) with each other and with the dataframe containing salary data.
    Assigns batting performance data column names to the year to which the data applies.
    
    Import requirements:
    import pandas as pd
    import numpy as np
    import pickle
    
    Requires a pickle file contaiing the returned dataframe of the fuction pickle_3yr_data (from above)
    Requires functions clean_batting_data and clean_salary_data (from above)
    
    Returns:
    Pandas dataframe with one row for each player that contains 2018 salary and batting performance data for each
    of three years, 2015, 2016 and 2017.
    
    '''
    
    # Read source data from pickle file
    with open("batting_value_pickle.pkl", "rb") as f:
        pickle_object = pickle.load(f)
        
    # Assign dataframe names to years to which the data applies
    df_list = pickle_object[0]
    df_batting_2015 = df_list[0]
    df_batting_2015 = clean_batting_data(df_batting_2015)
    df_batting_2016 = df_list[1]
    df_batting_2016 = clean_batting_data(df_batting_2016)
    df_batting_2017 = df_list[2]
    
    # Clean batting data
    df_batting_2017 = clean_batting_data(df_batting_2017)
    
    # Clean salary data
    df_salary = clean_salary_data()
    
    # Merge 2018 salary data and the batting data dataframes for each of three years
    # Assign column names to batting data in merged dataframe that identifies year to which the data applies
    df_combined = pd.merge(df_salary, df_batting_2015, how = 'left', on='Name')
    df_combined = df_combined.merge(df_batting_2016, on='Name', how='left', suffixes=("_2015", "_2016"))
    df_combined = df_combined.merge(df_batting_2017, on='Name', how='left')
    
    # Drop rows with any missing values
    df_combined.dropna(axis=0, how='any', inplace=True)
    
    # Return merged dataframe
    return df_combined

def calculate_average_features(df_combined):
    
    '''Builds Pandas dataframe with one row for each player that contains the 2018 salary data
    and batting performace data that is the average of batting performance over three years 2015, 2016, 2017.
    
    Import requirements:
    import pandas as pd
    import numpy as np
    
    Input parameter:
    Dataframe returned by function combine_data (from_above)
    
    Returns:
    Pandas dataframe with 2018 salary data and average batting performance data over three years 2015, 2016, 2017
    for each player.
    
    '''
    df_avg_features = df_combined.loc[:, ['Name', 'salary']]
    df_avg_features['avg_age'] = (df_combined.loc[:, 'Age_2016']).astype('int')
    df_avg_features['avg_games'] = ((df_combined['G_2015'] + df_combined['G_2016'] + df_combined['G']) / 3.0).astype('int')
    df_avg_features['avg_PA'] = ((df_combined['PA_2015'] + df_combined['PA_2016'] + df_combined['PA']) / 3.0).astype('int')
    df_avg_features['avg_AB'] = ((df_combined['AB_2015'] + df_combined['AB_2016'] + df_combined['AB']) / 3.0).astype('int')
    df_avg_features['avg_R'] = ((df_combined['R_2015'] + df_combined['R_2016'] + df_combined['R']) / 3.0).astype('int')
    df_avg_features['avg_H'] = ((df_combined['H_2015'] + df_combined['H_2016'] + df_combined['H']) / 3.0).astype('int')
    df_avg_features['avg_2B'] = ((df_combined['2B_2015'] + df_combined['2B_2016'] + df_combined['2B']) / 3.0).astype('int')
    df_avg_features['avg_3B'] = ((df_combined['3B_2015'] + df_combined['3B_2016'] + df_combined['3B']) / 3.0).astype('int')
    df_avg_features['avg_HR'] = ((df_combined['HR_2015'] + df_combined['HR_2016'] + df_combined['HR']) / 3.0).astype('int')
    df_avg_features['avg_RBI'] = ((df_combined['RBI_2015'] + df_combined['RBI_2016'] + df_combined['RBI']) / 3.0).astype('int')
    df_avg_features['avg_BB'] = ((df_combined['BB_2015'] + df_combined['BB_2016'] + df_combined['BB']) / 3.0).astype('int')
    df_avg_features['avg_SO'] = ((df_combined['SO_2015'] + df_combined['SO_2016'] + df_combined['SO']) / 3.0).astype('int')
    df_avg_features['avg_BA'] = ((df_combined['BA_2015'] + df_combined['BA_2016'] + df_combined['BA']) / 3.0)
    df_avg_features['avg_OBP'] = ((df_combined['OBP_2015'] + df_combined['OBP_2016'] + df_combined['OBP']) / 3.0)
    df_avg_features['avg_SLG'] = ((df_combined['SLG_2015'] + df_combined['SLG_2016'] + df_combined['SLG']) / 3.0)
    df_avg_features['avg_OPS'] = ((df_combined['OPS_2015'] + df_combined['OPS_2016'] + df_combined['OPS']) / 3.0)
    df_avg_features['avg_OPS+'] = ((df_combined['OPS+_2015'] + df_combined['OPS+_2016'] + df_combined['OPS+']) / 3.0)
    df_avg_features['avg_TB'] = ((df_combined['TB_2015'] + df_combined['TB_2016'] + df_combined['TB']) / 3.0).astype('int')
    df_avg_features['avg_GDP'] = ((df_combined['GDP_2015'] + df_combined['GDP_2016'] + df_combined['GDP']) / 3.0).astype('int')
    df_avg_features['avg_HBP'] = ((df_combined['HBP_2015'] + df_combined['HBP_2016'] + df_combined['HBP']) / 3.0).astype('int')
    df_avg_features['avg_SH'] = ((df_combined['SH_2015'] + df_combined['SH_2016'] + df_combined['SH']) / 3.0).astype('int')
    df_avg_features['avg_SF'] = ((df_combined['SF_2015'] + df_combined['SF_2016'] + df_combined['SF']) / 3.0).astype('int')
    df_avg_features['avg_IBB'] = ((df_combined['IBB_2015'] + df_combined['IBB_2016'] + df_combined['IBB']) / 3.0).astype('int')
    return df_avg_features

def simple_features(df_avg_features):
    
    '''
    Selects simple batting performance statistics (directly measured rather than calculated from other measures)
    
    Import requirements:
    import pandas as pd
    import numpy as np
    
    Input parameter:
    Dataframe returned by function calculate_average_features (from_above)
    
    Returns:
    Pandas dataframe with 2018 salary data and average batting performance data over three years 2015, 2016, 2017
    for simple batting performance statistics for each player.
    '''
    
    simple_columns = ['salary', 'avg_R', 'avg_H', 'avg_RBI', 'avg_BB', 'avg_SO',
        'avg_GDP', 'avg_HBP', 'avg_SH', 'avg_SF', 'avg_IBB']
    df_simple = df_avg_features[simple_columns]
    return df_simple

def linear_regression_model(df):
    
    '''Splits dataframe contiaining salary and batting performance data into training and test datasets, 
    log transforms salary data, scales batting data and defines linear regression model for the data.
    
    Import requirements:
    import pandas as pd
    import numpy as np
    from sklearn.linear_model import LinearRegression
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import train_test_split as tts
    
    Input parameter:
    Dataframe returned by function simple_features or dataframe with similar structure including player
    salaries and batting performance data.
    
    Returns:
    Linear regression model, training and test data
    '''
    X = df[[x for x in df.columns if x != 'salary']]
    y = df['salary']
    X_train, X_test, y_train, y_test = tts(X, y, test_size=0.3, random_state=42)
    y_train = np.log(y_train)
    y_test = np.log(y_test)
    ssX = StandardScaler()
    ssX.fit(X_train)
    X_train = ssX.transform(X_train)
    X_test = ssX.transform(X_test)
    model= LinearRegression()
    model = model.fit(X_train, y_train)
    return model, X_train, X_test, y_train, y_test

def mse(model, X, y):
    
    '''
    Calculates mean squared error.
    
    Import requirements:
    import pandas as pd
    import numpy as np
    from sklearn.metrics import mean_squared_error

    Input parameters:
    Scikit Learn regression model 
    Feature data (X)
    Target data (y)
    (as returned by function linear_regression model from above)
    
    '''
    y_predict = model.predict(X)
    mse = mean_squared_error(y, y_predict)
    return mse

def mae(model, X, y):
    '''
    from sklearn.metrics import mean_absolute_error'''
    y_exp = np.exp(y)
    y_predict = model.predict(X)
    y_predict_exp = np.exp(y_predict)
    mae = mean_absolute_error(y_exp, y_predict_exp)
    return mae

In [3]:
pickle_3yr_data()

In [4]:
df_combined = combine_data()
df_average_features = calculate_average_features(df_combined)
df_simple = simple_features(df_average_features)
model, X_train, X_test, y_train, y_test = linear_regression_model(df_simple)
mse_test = mse(model, X_test, y_test)
mse_train = mse(model, X_train, y_train)
mae = mae(model, X_test, y_test)
print("train mean squared error = ", mse_train, "test mean squared error = ", mse_test, "mean_absolute_error = ", mae )

train mean squared error =  0.38674348867827474 test mean squared error =  0.37127053929629716 mean_absolute_error =  4145517.9500637283
